In [0]:
# To download Already any Dataset in file.iF need to delete dateset then uncomment the lines of code
# dbutils.fs.rm("/FileStore/tables/bronze", recurse=True)
# dbutils.fs.rm("/FileStore/tables/silver", recurse=True)
# dbutils.fs.rm("/FileStore/tables/gold", recurse=True)
# dbutils.fs.rm("/FileStore/tables/predictive_maintenance-1.csv", recurse=True)
# dbutils.fs.rm("/FileStore/tables/predictive_maintenance.csv", recurse=True)

In [0]:
# 1. Environment Setup
# Install required packages
%pip install pyspark==3.5.0 delta-spark==3.0.0 mlflow scikit-learn

# Restart Python to apply changes
dbutils.library.restartPython()


Python interpreter will be restarted.
  Using cached pyspark-3.5.0-py2.py3-none-any.whl
  Using cached delta_spark-3.0.0-py3-none-any.whl (21 kB)
  Using cached mlflow-2.22.0-py3-none-any.whl (29.0 MB)
  Using cached py4j-0.10.9.7-py2.py3-none-any.whl (200 kB)
  Using cached importlib_metadata-8.7.0-py3-none-any.whl (27 kB)
  Using cached sqlalchemy-2.0.41-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.2 MB)
  Using cached mlflow_skinny-2.22.0-py3-none-any.whl (6.3 MB)
  Using cached graphene-3.4.3-py2.py3-none-any.whl (114 kB)
  Using cached docker-7.1.0-py3-none-any.whl (147 kB)
  Using cached flask-3.1.1-py3-none-any.whl (103 kB)
  Using cached gunicorn-23.0.0-py3-none-any.whl (85 kB)
  Using cached markdown-3.8-py3-none-any.whl (106 kB)
  Using cached alembic-1.16.1-py3-none-any.whl (242 kB)
  Using cached GitPython-3.1.44-py3-none-any.whl (207 kB)
  Using cached sqlparse-0.5.3-py3-none-any.whl (44 kB)
  Using cached opentelemetry_api-1.33.1-py3-none-any.whl (65 kB)
  

In [0]:
# 2. Data Ingestion (Bronze Layer)
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Initialize Spark with Delta Lake
spark = SparkSession.builder \
    .appName("PredictiveMaintenance") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .getOrCreate()

# Load your dataset
df = spark.read.csv("/FileStore/tables/predictive_maintenance.csv", header=True, inferSchema=True)

# Clean column names (remove spaces and special characters)
clean_columns = [c.replace(" ", "_").replace("[", "").replace("]", "") for c in df.columns]
clean_df = df.toDF(*clean_columns)

# Save to Bronze Layer
bronze_path = "dbfs:/FileStore/tables/bronze/equipment_raw"
clean_df.write.format("delta").mode("overwrite").save(bronze_path)

# Verify
display(spark.read.format("delta").load(bronze_path).limit(5))

UDI Product_ID Type Air_temperature_K Process_temperature_K Rotational_speed_rpm Torque_Nm Tool_wear_min Target Failure_Type 1 M14860 M 298.1 308.6 1551 42.8 0 0 No Failure 2 L47181 L 298.2 308.7 1408 46.3 3 0 No Failure 3 L47182 L 298.1 308.5 1498 49.4 5 0 No Failure 4 L47183 L 298.2 308.6 1433 39.5 7 0 No Failure 5 L47184 L 298.2 308.7 1408 40.0 9 0 No Failure

In [0]:
# 3. Data Cleaning (Silver Layer)
# Read from Bronze Layer
silver_df = spark.read.format("delta").load(bronze_path)

# Data cleaning specific to your dataset
silver_df = (silver_df
    .filter(
        (col("Air_temperature_K") > 0) &
        (col("Process_temperature_K") > 0) &
        (col("Rotational_speed_rpm") > 0)
    )
    .dropDuplicates(["UDI"])
    .na.fill(0, subset=["Tool_wear_min"])
)

# Save to Silver Layer
silver_path = "dbfs:/FileStore/tables/silver/equipment_clean"
silver_df.write.format("delta").mode("overwrite").save(silver_path)

# Verify
display(spark.read.format("delta").load(silver_path).limit(5))


UDI Product_ID Type Air_temperature_K Process_temperature_K Rotational_speed_rpm Torque_Nm Tool_wear_min Target Failure_Type 1 M14860 M 298.1 308.6 1551 42.8 0 0 No Failure 2 L47181 L 298.2 308.7 1408 46.3 3 0 No Failure 3 L47182 L 298.1 308.5 1498 49.4 5 0 No Failure 4 L47183 L 298.2 308.6 1433 39.5 7 0 No Failure 5 L47184 L 298.2 308.7 1408 40.0 9 0 No Failure

In [0]:
# 4. Feature Engineering (Gold Layer)

from pyspark.sql.window import Window
from pyspark.sql.functions import avg, max, min, stddev

# Define window for rolling features
window = Window.partitionBy("Product_ID").orderBy("Process_temperature_K").rowsBetween(-5, 0)

# Create advanced features
gold_df = (silver_df
    .withColumn("avg_temp_5d", avg(col("Air_temperature_K")).over(window))
    .withColumn("max_rpm_5d", max(col("Rotational_speed_rpm")).over(window))
    .withColumn("torque_variation", stddev(col("Torque_Nm")).over(window))
    .withColumn("tool_wear_rate", 
               (col("Tool_wear_min") - min(col("Tool_wear_min")).over(window)) / 
               (max(col("Process_temperature_K")).over(window) - min(col("Process_temperature_K")).over(window))
)
)

# Save to Gold Layer
gold_path = "dbfs:/FileStore/tables/gold/equipment_features"
gold_df.write.format("delta").mode("overwrite").save(gold_path)

# Show engineered features
display(gold_df.select(
    "Product_ID", "Type", "avg_temp_5d", "max_rpm_5d", 
    "torque_variation", "tool_wear_rate", "Target"
).limit(10))

Product_ID Type avg_temp_5d max_rpm_5d torque_variation tool_wear_rate Target H29424 H 298.4 1782 null null 0 H29425 H 298.6 1423 null null 0 H29432 H 298.8 1306 null null 0 H29434 H 298.9 1375 null null 0 H29441 H 299.1 1811 null null 0 H29452 H 298.9 1379 null null 0 H29457 H 298.8 1372 null null 0 H29462 H 298.8 1425 null null 0 H29466 H 298.8 1497 null null 0 H29481 H 298.8 1466 null null 0

In [0]:
# 5. Model Training
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import VectorAssembler, StringIndexer, OneHotEncoder
from pyspark.ml import Pipeline
from pyspark.sql.functions import col

# 1. First check for nulls in critical columns
null_counts = gold_df.select([
    col(c).isNull().cast("int").alias(c) 
    for c in ["avg_temp_5d", "max_rpm_5d", "torque_variation", "tool_wear_rate", "Type", "Target"]
]).groupBy().sum().collect()[0].asDict()

print("Null value counts:")
for col_name, count in null_counts.items():
    print(f"{col_name}: {count}")

# 2. Define features (separate numeric and categorical)
numeric_features = [
    "avg_temp_5d", 
    "max_rpm_5d",
    "torque_variation",
    "tool_wear_rate"
]
categorical_features = ["Type"]

# 3. Create preprocessing stages with null handling
type_indexer = StringIndexer(
    inputCol="Type",
    outputCol="Type_index",
    handleInvalid="keep"  # Keeps nulls as extra category
)

type_encoder = OneHotEncoder(
    inputCol="Type_index",
    outputCol="Type_encoded",
    handleInvalid="keep"
)

assembler = VectorAssembler(
    inputCols=numeric_features + ["Type_encoded"],
    outputCol="features",
    handleInvalid="keep"  # Keeps rows with nulls
)

# 4. Split data (after verifying no nulls in Target)
train_df, test_df = gold_df.filter(col("Target").isNotNull()).randomSplit([0.8, 0.2], seed=42)

# 5. Define model
rf = RandomForestClassifier(
    featuresCol="features",
    labelCol="Target",
    numTrees=100,
    maxDepth=5,
    seed=42
)

# 6. Create and train pipeline
pipeline = Pipeline(stages=[
    type_indexer,
    type_encoder,
    assembler,
    rf
])

# 7. Train model with try-catch
try:
    model = pipeline.fit(train_df)
    
    # Test pipeline
    test_predictions = model.transform(test_df.limit(5))
    display(test_predictions.select("features", "prediction", "Target"))
    print("✅ Pipeline executed successfully!")
except Exception as e:
    print(f"❌ Pipeline error: {str(e)}")
    print("Try handling nulls in your data first")

Null value counts:
sum(avg_temp_5d): 0
sum(max_rpm_5d): 0
sum(torque_variation): 10000
sum(tool_wear_rate): 10000
sum(Type): 0
sum(Target): 0


features prediction Target Map(vectorType -> dense, length -> 8, values -> List(298.1, 1498.0, NaN, NaN, 1.0, 0.0, 0.0, 0.0)) 0.0 0 Map(vectorType -> dense, length -> 8, values -> List(298.1, 1558.0, NaN, NaN, 1.0, 0.0, 0.0, 0.0)) 0.0 0 Map(vectorType -> dense, length -> 8, values -> List(298.3, 1667.0, NaN, NaN, 0.0, 1.0, 0.0, 0.0)) 0.0 0 Map(vectorType -> dense, length -> 8, values -> List(298.6, 1742.0, NaN, NaN, 0.0, 1.0, 0.0, 0.0)) 0.0 0 Map(vectorType -> dense, length -> 8, values -> List(298.9, 1632.0, NaN, NaN, 0.0, 1.0, 0.0, 0.0)) 0.0 0

✅ Pipeline executed successfully!


In [0]:
# 6. Model Evaluation

from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Make predictions
predictions = model.transform(test_df)

# Evaluate
evaluator = BinaryClassificationEvaluator(
    labelCol="Target",
    metricName="areaUnderROC"
)
auc = evaluator.evaluate(predictions)

print(f"Model AUC: {auc:.2%}")

# Confusion matrix
display(predictions.groupBy("Target", "prediction").count())

Model AUC: 84.74%


Target prediction count 1 0.0 82 0 0.0 1839

In [0]:
# 7. MLflow Tracking

import mlflow
import matplotlib.pyplot as plt
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# 1. FIRST define your feature columns (must match training exactly)
feature_cols = [
    "avg_temp_5d", 
    "max_rpm_5d",
    "torque_variation",
    "tool_wear_rate",
    "Type_encoded"  # This should match what's in your pipeline
]

# 2. Calculate AUC if not already done
evaluator = BinaryClassificationEvaluator(
    labelCol="Target",
    metricName="areaUnderROC"
)
predictions = model.transform(test_df)
auc = evaluator.evaluate(predictions)

# 3. MLflow Tracking
mlflow.set_experiment("/Shared/Predictive_Maintenance")

with mlflow.start_run():
    # Log parameters (now feature_cols is defined)
    mlflow.log_param("num_trees", 100)
    mlflow.log_param("max_depth", 5)
    mlflow.log_param("features", feature_cols)
    
    # Log metrics
    mlflow.log_metric("auc", auc)
    
    # Log model
    mlflow.spark.log_model(model, "model")
    
    # Log feature importance plot
    try:
        rf_model = model.stages[-1]  # Get RandomForest model from pipeline
        importance = rf_model.featureImportances.toArray()
        
        # Verify lengths match
        if len(feature_cols) != len(importance):
            print(f"Warning: Mismatch in lengths - features: {len(feature_cols)}, importance: {len(importance)}")
            # Handle mismatch by pairing and sorting
            features_importance = list(zip(feature_cols, importance))
            features_importance.sort(key=lambda x: x[1])  # Sort by importance
            feature_cols = [x[0] for x in features_importance]
            importance = [x[1] for x in features_importance]
        
        # Create plot
        plt.figure(figsize=(10, 6))
        plt.barh(feature_cols, importance)
        plt.title("Feature Importance")
        plt.tight_layout()
        plt.savefig("feature_importance.png")
        mlflow.log_artifact("feature_importance.png")
        plt.close()
    except Exception as e:
        print(f"⚠️ Could not log feature importance: {str(e)}")
    
    print(f"✅ Model logged with AUC: {auc:.2%}")
    print(f"Run ID: {mlflow.active_run().info.run_id}")

2025/05/24 16:53:19 INFO mlflow.spark: Inferring pip requirements by reloading the logged model from the databricks artifact repository, which can be time-consuming. To speed up, explicitly specify the conda_env or pip_requirements when calling log_model().
2025/05/24 16:53:43 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: dbfs:/databricks/mlflow-tracking/4289069952042785/72b596fbbecf4cfe8ae2bb6962165584/artifacts/model/sparkml, flavor: spark). Fall back to return ['pyspark==3.5.0', 'pandas<2']. Set logging level to DEBUG to see the full traceback. 


✅ Model logged with AUC: 84.74%
Run ID: 72b596fbbecf4cfe8ae2bb6962165584


In [0]:
# 8. Model Deployment
import mlflow
from pyspark.sql.functions import avg, max, min, stddev
from pyspark.sql.window import Window

# 1. Load the trained pipeline
run_id = mlflow.search_runs(
    experiment_names=["/Shared/Predictive_Maintenance"],
    order_by=["start_time DESC"]
).iloc[0].run_id

model_uri = f"runs:/{run_id}/model"
loaded_pipeline = mlflow.spark.load_model(model_uri)

# 2. Prepare new data with ALL required raw features
new_data = spark.createDataFrame([{
    "Type": "M",
    "Air_temperature_K": 298.5,
    "Process_temperature_K": 309.0,
    "Rotational_speed_rpm": 1750,
    "Torque_Nm": 30.2,
    "Tool_wear_min": 45,
    "Product_ID": "M14860"  # Needed for window calculations
}])

# 3. Recreate the exact same feature engineering as training
window = Window.partitionBy("Product_ID").orderBy("Process_temperature_K").rowsBetween(-5, 0)

new_data = (new_data
    .withColumn("avg_temp_5d", avg(col("Air_temperature_K")).over(window))
    .withColumn("max_rpm_5d", max(col("Rotational_speed_rpm")).over(window))
    .withColumn("torque_variation", stddev(col("Torque_Nm")).over(window))
    .withColumn("tool_wear_rate", 
               (col("Tool_wear_min") - min(col("Tool_wear_min")).over(window)) / 
               (max(col("Process_temperature_K")).over(window) - min(col("Process_temperature_K")).over(window)))
)

# 4. Make prediction
predictions = loaded_pipeline.transform(new_data)
display(predictions.select("prediction", "probability"))

2025/05/24 16:53:51 INFO mlflow.spark: URI 'runs:/72b596fbbecf4cfe8ae2bb6962165584/model/sparkml' does not point to the current DFS.
2025/05/24 16:53:51 INFO mlflow.spark: File 'runs:/72b596fbbecf4cfe8ae2bb6962165584/model/sparkml' not found on DFS. Will attempt to upload the file.
2025/05/24 16:53:56 INFO mlflow.spark: Copied SparkML model to /tmp/mlflow/0d78be7f-e8ea-4f92-9b0b-9e5cd2734c2c


prediction probability 0.0 Map(vectorType -> dense, length -> 2, values -> List(0.9759985772399128, 0.024001422760087158))